Ciclismo
Tiempo de muestreo: 2661 [s]
Número de muestras: 156560

Número de muestras en un segundo: 58.835024

Para el primer archivo:

Debo elegir una secuencia, que debe ser igual al número de muestras por minuto, después debo normalizar los valores de los sensores empleando MinMax()

Posteriormente debo seleccionar los datos de entrenamiento y ¿Creamos las secuencias con los datos de los sensores? ¿No debería de seleccionar a Y como la etiqueta de clasificación Bicycling?

In [2]:
#Manipulación de datos, generación y modificación de dataframes
import pandas as pd
# Importa MinMaxScaler de sklearn para escalar características
from sklearn.preprocessing import MinMaxScaler
#Operaciones matemáticas
import numpy as np
# Importa la biblioteca de PyTorch para cálculos numéricos y entrenamiento de modelos de aprendizaje profundo
import torch
# Importa nn y optim de torch para construir y optimizar redes neuronales
from torch import nn, optim

In [3]:
archivo_entrada='HuGaDB_v1_bicycling_01_00.txt'
archivo_salida = 'Ciclismo_P01'

# Leer todas las líneas del archivo de entrada
with open(archivo_entrada, 'r') as f:
  lineas = f.readlines()

    # Excluir las tres primeras líneas
lineas_sin_tres_primeras = lineas[3:]

    # Escribir las líneas restantes en el archivo de salida
with open(archivo_salida, 'w') as f:
  f.writelines(lineas_sin_tres_primeras)

# Leer el contenido del archivo de salida y convertirlo a un DataFrame
df = pd.read_csv(archivo_salida, sep='\t', header=0)

# Mostrar el DataFrame
df

,acc_rf_x,acc_rf_y,acc_rf_z,gyro_rf_x,gyro_rf_y,gyro_rf_z,acc_rs_x,acc_rs_y,acc_rs_z,gyro_rs_x,...,gyro_ls_z,acc_lt_x,acc_lt_y,acc_lt_z,gyro_lt_x,gyro_lt_y,gyro_lt_z,EMG_r,EMG_l,act
0,-14392,-96,10628,138,-388,492,-13980,-1408,-6808,-519,...,183,-11752,2228,9120,261,277,522,130,125,9
1,-11248,164,7744,169,699,429,-14044,-2608,-6992,-214,...,63,-13920,3292,8916,132,86,581,136,124,9
2,-15892,-1952,9324,-217,709,360,-13336,-3564,-7192,-326,...,69,-12756,2608,8796,174,-96,641,127,127,9
3,-13020,892,8604,-297,-62,170,-13832,-3220,-8068,-482,...,155,-13900,4236,8312,88,-63,553,122,127,9
4,-12780,-948,10744,-301,-1359,39,-13680,-2888,-6068,-889,...,-146,-12428,2596,8448,166,66,583,122,125,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8862,-22292,1944,2356,4170,7649,7617,-17820,-4640,-8456,135,...,-3608,-1508,-1188,1908,-407,-5415,2324,122,144,9
8863,-22428,5292,5204,2678,5379,5434,-20172,-2864,-6400,1496,...,-3247,-580,-2140,1988,837,-2803,1683,120,131,9
8864,-25484,-388,12520,6228,6605,6589,-22460,-4828,-4252,-2783,...,-3496,-1764,-380,3444,1672,-568,1471,128,120,9
8865,-16140,-3892,5920,5414,9979,8453,-22008,-820,-6416,1041,...,-3842,-4256,280,6032,2075,1275,1231,134,101,9


In [4]:
# Define el tamaño del conjunto de datos de prueba
test_data_size = 1773

'''Recuerda 80% entrenamiento 20% prueba'''

# Asigna los valores del DataFrame, excepto los últimos 1773, a train_data
train_data = df[:-test_data_size].values

# Asigna los valores de los últimos 1773 registros del DataFrame a test_data
test_data = df[-test_data_size:].values

print("Train data: \n")
print(train_data)
print('\nTest data: \n')
print(test_data)

Train data: 

[[-14392    -96  10628 ...    130    125      9]
 [-11248    164   7744 ...    136    124      9]
 [-15892  -1952   9324 ...    127    127      9]
 ...
 [ -7012   7568  -3280 ...    119    123      9]
 [ -3564   1216   1128 ...    118    124      9]
 [ -6236  -4392   1852 ...    131    123      9]]

Test data: 

[[ -7940   2956   2772 ...    116    128      9]
 [ -7656   2612   3536 ...     86    127      9]
 [-10644   5944   4620 ...    134    128      9]
 ...
 [-25484   -388  12520 ...    128    120      9]
 [-16140  -3892   5920 ...    134    101      9]
 [-20952   4768   5792 ...    126    119      9]]


que sea por participantes
80% entrenamiento y 20%prueba

In [5]:
# Crea una instancia del MinMaxScaler
scaler = MinMaxScaler()

# Ajusta el scaler a los datos de entrenamiento
scaler = scaler.fit(train_data)

# Transforma los datos de entrenamiento utilizando el scaler ajustado
train_data = scaler.transform(train_data)

# Transforma los datos de prueba utilizando el mismo scaler ajustado
test_data = scaler.transform(test_data)

print("Train data: \n")
print(train_data)
print(f'\n  Dimensiones: {train_data.shape}')
print('\nTest data: \n')
print(test_data)
print(f'\n  Dimensiones: {test_data.shape}')

Train data: 

[[0.29129415 0.49854276 0.56456149 ... 0.45303867 0.6        0.        ]
 [0.34113246 0.50251011 0.50783785 ... 0.48618785 0.59354839 0.        ]
 [0.26751633 0.47022202 0.53891391 ... 0.43646409 0.61290323 0.        ]
 ...
 [0.40828102 0.61548791 0.29101351 ... 0.39226519 0.58709677 0.        ]
 [0.4629383  0.5185626  0.37771178 ... 0.38674033 0.59354839 0.        ]
 [0.42058208 0.43299001 0.39195169 ... 0.45856354 0.58709677 0.        ]]

  Dimensiones: (7094, 39)

Test data: 

[[0.39357048 0.5451133  0.41004661 ... 0.37569061 0.61935484 0.        ]
 [0.39807241 0.53986419 0.42507326 ... 0.20994475 0.61290323 0.        ]
 [0.35070699 0.59070726 0.4463938  ... 0.47513812 0.61935484 0.        ]
 ...
 [0.11546509 0.49408713 0.60177409 ... 0.44198895 0.56774194 0.        ]
 [0.26358506 0.44061952 0.47196271 ... 0.47513812 0.44516129 0.        ]
 [0.18730581 0.57276265 0.46944515 ... 0.43093923 0.56129032 0.        ]]

  Dimensiones: (1773, 39)


In [6]:
def create_sequences(data, seq_length):
    # Inicializa dos listas vacías para almacenar las secuencias y sus correspondientes etiquetas
    xs = [] #Inicializa una lista vacía xs que almacenará las secuencias de entrada.
    ys = [] #Inicializa una lista vacía ys que almacenará las etiquetas correspondientes a cada secuencia.

    # Itera sobre el rango del tamaño de los datos menos la longitud de la secuencia menos uno
    for i in range(len(data) - seq_length - 1):
        # Toma una subsecuencia de longitud seq_length desde el índice i
        x = data[i:(i + seq_length)]
        # Toma el valor siguiente a la secuencia como la etiqueta
        y = data[i + seq_length]
        # Añade la secuencia a la lista xs
        xs.append(x)
        # Añade la etiqueta a la lista ys
        ys.append(y)

    # Convierte las listas xs y ys en arrays de numpy y los retorna
    return np.array(xs), np.array(ys)

In [7]:
# Define la longitud de la secuencia
seq_length = 1000

# Crea las secuencias y las etiquetas para los datos de entrenamiento utilizando la función create_sequences
X_train, y_train = create_sequences(train_data, seq_length)

# Crea las secuencias y las etiquetas para los datos de prueba utilizando la función create_sequences
X_test, y_test = create_sequences(test_data, seq_length)

# Convierte las secuencias de entrenamiento (X_train) a tensores de PyTorch de tipo float
X_train = torch.from_numpy(X_train).float()

# Convierte las etiquetas de entrenamiento (y_train) a tensores de PyTorch de tipo float
y_train = torch.from_numpy(y_train).float()

# Convierte las secuencias de prueba (X_test) a tensores de PyTorch de tipo float
X_test = torch.from_numpy(X_test).float()

# Convierte las etiquetas de prueba (y_test) a tensores de PyTorch de tipo float
y_test = torch.from_numpy(y_test).float()

print("X Train data: \n")
#print(X_train)
print(f'\n  Dimensiones: {X_train.shape}')
print("\nY Train data: \n")
#print(y_train)
print(f'\n  Dimensiones: {y_train.shape}')
print('\n X Test data: \n')
#print(X_test)
print(f'\n  Dimensiones: {X_test.shape}')
print("\nY Test data: \n")
#print(y_test)
print(f'\n  Dimensiones: {y_test.shape}')


X Train data: 


  Dimensiones: torch.Size([6093, 1000, 39])

Y Train data: 


  Dimensiones: torch.Size([6093, 39])

 X Test data: 


  Dimensiones: torch.Size([772, 1000, 39])

Y Test data: 


  Dimensiones: torch.Size([772, 39])


In [8]:
# Define una clase BatterySignalPredictor que hereda de nn.Module
class BatterySignalPredictor(nn.Module):

  # Método de inicialización de la clase
  def __init__(self, n_features, n_hidden, seq_len, n_layers=2):
    super(BatterySignalPredictor, self).__init__()

    # Inicializa las variables de instancia
    self.n_hidden = n_hidden
    self.seq_len = seq_len
    self.n_layers = n_layers

    # Define una capa LSTM
    self.lstm = nn.LSTM(
      input_size=n_features,  # Número de características de entrada
      hidden_size=n_hidden,   # Número de características en el estado oculto
      num_layers=n_layers     # Número de capas LSTM
    )

    # Define una capa lineal
    self.linear = nn.Linear(in_features=n_hidden, out_features=1)

  # Método para reiniciar el estado oculto, limpiar información previa.
  def reset_hidden_state(self):
    self.hidden = (
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden),  # Inicializa el estado oculto con ceros
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden)   # Inicializa el estado de la celda con ceros
    )

  # Método de propagación hacia adelante
  '''NO ENTIENDO MUY BIEN LO DE self.lstm'''
  def forward(self, sequences):
    # Pasa las secuencias por la capa LSTM
    lstm_out, self.hidden = self.lstm(
      sequences.view(len(sequences), self.seq_len, -1),  # Redimensiona las secuencias
      self.hidden  # Proporciona el estado oculto inicial
    )
    # Toma el último paso de tiempo de la salida LSTM
    last_time_step = lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]
    # Pasa la última salida de tiempo por la capa lineal
    y_pred = self.linear(last_time_step)
    return y_pred  # Devuelve la predicción

In [9]:
def train_model(
  model,
  train_data,
  train_labels,
  test_data=None,
  test_labels=None
):
  loss_fn = torch.nn.MSELoss(reduction='sum')

  optimiser = torch.optim.Adam(model.parameters(), lr=1e-2)
  num_epochs = 300

  train_hist = np.zeros(num_epochs)
  test_hist = np.zeros(num_epochs)

  for t in range(num_epochs):
    model.reset_hidden_state()

    y_pred = model(X_train)

    loss = loss_fn(y_pred.float(), y_train)

    if test_data is not None:
      with torch.no_grad():
        y_test_pred = model(X_test)
        test_loss = loss_fn(y_test_pred.float(), y_test)
      test_hist[t] = test_loss.item()

      if t % 10 == 0:
        print(f'Epoch {t} train loss: {loss.item()} test loss: {test_loss.item()}')
    elif t % 10 == 0:
      print(f'Epoch {t} train loss: {loss.item()}')

    train_hist[t] = loss.item()

    optimiser.zero_grad()

    loss.backward()

    optimiser.step()

  return model.eval(), train_hist, test_hist

In [1]:
model = BatterySignalPredictor(
  n_features=39,
  n_hidden=700,
  seq_len=seq_length,
  n_layers=1
)
model, train_hist, test_hist = train_model(
  model,
  X_train,
  y_train,
  X_test,
  y_test
)

NameError: name 'BatterySignalPredictor' is not defined